In [1]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import os, pickle
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.cluster import KMeans

from sklearn.preprocessing import OneHotEncoder

from itertools import product

import sys, os

import trajectory as T                      # trajectory generation
import optimizer as O                       # stochastic gradient descent optimizer
import solver as S                          # MDP solver (value-iteration)
import plot as P


num_data = 355504


np.random.seed(66)

def to_interval(istr):
    c_left = istr[0]=='['
    c_right = istr[-1]==']'
    closed = {(True, False): 'left',
              (False, True): 'right',
              (True, True): 'both',
              (False, False): 'neither'
              }[c_left, c_right]
    left, right = map(pd.to_datetime, istr[1:-1].split(','))
    return pd.Interval(left, right, closed)

re_split = False
frac = [0.4,0.2,0.4]
assert np.sum(frac) == 1
frac = np.cumsum(frac)
print (frac)
data_save_path= 'data/'

def sliding(gs, window_size = 6):
    npr_l = []
    for g in gs:
        npr = np.concatenate([np.zeros([window_size-1, g.shape[1]]),g])
        npr_l.append(sliding_window_view(npr, (window_size, g.shape[1])).squeeze(1))
    return np.vstack(npr_l)

2023-12-03 13:33:18.696916: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[0.4 0.6 1. ]


## Baseline Data

In [2]:
with open('data/train_df', 'rb') as f:
    train_df = pickle.load(f)

with open('data/test_df', 'rb') as f:
    test_df = pickle.load(f)
    
with open('data/valid_df', 'rb') as f:
    valid_df = pickle.load(f)

In [3]:
data_non_normalized_df = pd.concat([train_df, valid_df, test_df], axis=0, ignore_index=False).head(num_data).copy()

#### standard normalization ####
normalize_features = ['creatinine', 'fraction_inspired_oxygen', 'lactate', 'urine_output',
                  'alanine_aminotransferase', 'asparate_aminotransferase',
                  'mean_blood_pressure', 'diastolic_blood_pressure',
                  'systolic_blood_pressure', 'gcs', 'partial_pressure_of_oxygen']
mu, std = (train_df[normalize_features]).mean().values,(train_df[normalize_features]).std().values

# Baseline Dataframe
train_df[normalize_features] = (train_df[normalize_features] - mu)/std
test_df[normalize_features] = (test_df[normalize_features] - mu)/std
valid_df[normalize_features] = (valid_df[normalize_features] - mu)/std




### create data matrix ####
X_train = train_df.loc[:,train_df.columns!='action']
y_train = train_df['action']

X_test = test_df.loc[:,test_df.columns!='action']
y_test = test_df['action']

X_valid = valid_df.loc[:, valid_df.columns!='action']
y_valid = valid_df['action']

In [4]:
X_df = pd.concat([X_train, X_valid, X_test], axis=0, ignore_index=True).copy()
y_df = pd.concat([y_train, y_valid, y_test], axis=0, ignore_index=True).copy()

In [5]:
data_df = pd.concat([train_df, valid_df, test_df], axis=0, ignore_index=False).copy()

## Observational Ambiguity Data

In [7]:
with open('data/obs_train_df', 'rb') as f:
    obs_train_df = pickle.load(f)

with open('data/obs_test_df', 'rb') as f:
    obs_test_df = pickle.load(f)
    
with open('data/obs_valid_df', 'rb') as f:
    obs_valid_df = pickle.load(f)

In [8]:
obs_data_non_normalized_df = pd.concat([obs_train_df, obs_valid_df, obs_test_df], axis=0, ignore_index=False).head(num_data).copy()
mu, std = (obs_train_df[normalize_features]).mean().values,(obs_train_df[normalize_features]).std().values

# Baseline Dataframe
obs_train_df[normalize_features] = (obs_train_df[normalize_features] - mu)/std
obs_test_df[normalize_features] = (obs_test_df[normalize_features] - mu)/std
obs_valid_df[normalize_features] = (obs_valid_df[normalize_features] - mu)/std


### create data matrix ####
obs_X_train = obs_train_df.loc[:,obs_train_df.columns!='action']
obs_y_train = obs_train_df['action']

obs_X_test = obs_test_df.loc[:,obs_test_df.columns!='action']
obs_y_test = obs_test_df['action']

obs_X_valid = obs_valid_df.loc[:, obs_valid_df.columns!='action']
obs_y_valid = obs_valid_df['action']

In [9]:
obs_X_df = pd.concat([obs_X_train, obs_X_valid, obs_X_test], axis=0, ignore_index=True).copy()
obs_y_df = pd.concat([obs_y_train, obs_y_valid, obs_y_test], axis=0, ignore_index=True).copy()

In [10]:
obs_data_df = pd.concat([obs_train_df, obs_valid_df, obs_test_df], axis=0, ignore_index=False).copy()

## Action Ambiguity Data

In [11]:
with open('data/act_train_df', 'rb') as f:
    act_train_df = pickle.load(f)

with open('data/act_test_df', 'rb') as f:
    act_test_df = pickle.load(f)
    
with open('data/act_valid_df', 'rb') as f:
    act_valid_df = pickle.load(f)

In [12]:
act_data_non_normalized_df = pd.concat([act_train_df, act_valid_df, act_test_df], axis=0, ignore_index=False).head(num_data).copy()
mu, std = (act_train_df[normalize_features]).mean().values,(act_train_df[normalize_features]).std().values

# Baseline Dataframe
act_train_df[normalize_features] = (act_train_df[normalize_features] - mu)/std
act_test_df[normalize_features] = (act_test_df[normalize_features] - mu)/std
act_valid_df[normalize_features] = (act_valid_df[normalize_features] - mu)/std


### create data matrix ####
act_X_train = act_train_df.loc[:,act_train_df.columns!='action']
act_y_train = act_train_df['action']

act_X_test = act_test_df.loc[:,act_test_df.columns!='action']
act_y_test = act_test_df['action']

act_X_valid = act_valid_df.loc[:, act_valid_df.columns!='action']
act_y_valid = act_valid_df['action']

In [13]:
act_X_df = pd.concat([act_X_train, act_X_valid, act_X_test], axis=0, ignore_index=True).copy()
act_y_df = pd.concat([act_y_train, act_y_valid, act_y_test], axis=0, ignore_index=True).copy()

In [14]:
act_data_df = pd.concat([act_train_df, act_valid_df, act_test_df], axis=0, ignore_index=False).copy()

## Baseline Clustering

In [28]:
num_clusters = 100

with open('data/clusters', 'rb') as f:
    kmeans = pickle.load(f)

/Users/jamin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 1.3.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [29]:
X_df['cluster'] = kmeans.labels_.copy()
data_df['cluster'] = kmeans.labels_.copy()
data_non_normalized_df['cluster'] = kmeans.labels_.copy()

## Observational Ambiguity Clustering

In [30]:
with open('data/obs_clusters', 'rb') as f:
    obs_kmeans = pickle.load(f)

In [31]:
obs_X_df['cluster'] = obs_kmeans.labels_.copy()
obs_data_df['cluster'] = obs_kmeans.labels_.copy()
obs_data_non_normalized_df['cluster'] = obs_kmeans.labels_.copy()

## Action Ambiguity Clustering

In [32]:
with open('data/act_clusters', 'rb') as f:
    act_kmeans = pickle.load(f)

/Users/jamin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 1.3.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [33]:
act_X_df['cluster'] = act_kmeans.labels_.copy()
act_data_df['cluster'] = act_kmeans.labels_.copy()
act_data_non_normalized_df['cluster'] = act_kmeans.labels_.copy()

## BC Model

In [ ]:
# Convert states and actions to one-hot encoding
state_encoder = OneHotEncoder(sparse=False, categories= [np.arange(num_clusters)])
action_encoder = OneHotEncoder(sparse=False, categories= [np.arange(4)])

# Baseline
states_onehot = state_encoder.fit_transform(X_df['cluster'].to_numpy().reshape(-1, 1))
actions_onehot = action_encoder.fit_transform(y_df.to_numpy().reshape(-1, 1))

# Observational Ambiguity
# states_onehot = state_encoder.fit_transform(obs_X_df['cluster'].to_numpy().reshape(-1, 1))
# actions_onehot = action_encoder.fit_transform(obs_y_df.to_numpy().reshape(-1, 1))

# Action Ambiguity
# states_onehot = state_encoder.fit_transform(act_X_df['cluster'].to_numpy().reshape(-1, 1))
# actions_onehot = action_encoder.fit_transform(act_y_df.to_numpy().reshape(-1, 1))


# # Define neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(states_onehot.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(actions_onehot.shape[1], activation='softmax')  # Output layer with softmax for discrete actions
])

# # Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

# # Train the model
model.fit(states_onehot, actions_onehot,  epochs=5, batch_size=128)

# # Evaluate the model
test_loss = model.evaluate(states_onehot, actions_onehot)
print("Test Loss:", test_loss)

In [ ]:
bc_policy = np.argmax(model.predict(state_encoder.transform(np.arange(num_clusters).reshape(-1, 1))), axis =1)
bc_policy